In [35]:
class Abalone():
    def __init__(self, sex, length, diameter, height, wholeHeight, shuckedWeight, visceraWeight, shellWeight, rings):
        self.sex=sex
        self.length=length
        self.diameter=diameter
        self.height=height
        self.wholeHeight=wholeHeight
        self.shuckedWeight=shuckedWeight
        self.visceraWeight=visceraWeight
        self.shellWeight=shellWeight
        self.rings=rings
        
    def __repr__(self):
        str = "Sex:{sex}\nLength:{length}\nDiameter:{diameter}\nHeigth:{height}\n"\
        "WholeHeight:{wholeHeight}\nShuckedWeight:{shuckedWeight}\n"\
        "VisceraWeight:{visceraWeight}\nShellWeight:{shellWeight}\nRings:{rings}\n"
        str = str.format(sex=self.sex, length=self.length, diameter = self.diameter,
                        height=self.height,wholeHeight=self.wholeHeight,shuckedWeight=self.shuckedWeight,
                         visceraWeight=self.visceraWeight, shellWeight=self.shellWeight, rings=self.rings)
        return str
            

In [36]:
def readAbaloneData():
    abalones = []
    try:
        with open("data/abalone.data","r") as f:
            for line in f.read().splitlines():
                sd = line.split(",") # split data
                abalones.append(Abalone(sd[0],sd[1],sd[2],sd[3],sd[4],sd[5],sd[6],sd[7],sd[8]))
    except Exception as e:
        print("Exception:",str(e))
        abalones = []
    return abalones
abalones = readAbaloneData()
print(abalones[0])

Sex:M
Length:0.455
Diameter:0.365
Heigth:0.095
WholeHeight:0.514
ShuckedWeight:0.2245
VisceraWeight:0.101
ShellWeight:0.15
Rings:15



In [ ]:
def build_dt(X, y, attribute_types, options):
    